# 10.9 Lab: Deep Learning

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score,confusion_matrix,roc_curve, auc, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import json
import tensorflow.keras as tk
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.utils.np_utils import to_categorical
from keras.regularizers import l1, l2

%matplotlib inline


In [ ]:
""" 
In Python, we use keras as the DL interface with backend Tensorflow.
pip install keras 
"""

## 10.9.1 A Single Layer Network on the Hitters Data

In [ ]:
# In this exercise, we will use Hitters data set to predict the salary of a player
# I will skip the linear regression and Lasso part since we covered them in previous chapters

In [ ]:
# read the data and take a look at the data and split it into train and test 
# I copied the code from chapter (because of laziness :- )
Hitters = pd.read_csv('data/Hitters.csv', header=0, na_values='NA')
Hitters = Hitters.dropna().reset_index(drop=True) # drop the observation with NA values and reindex the obs from 0
dummies = pd.get_dummies(Hitters[['League', 'Division', 'NewLeague']])

y = Hitters.Salary  # the response variable 
X_prep = Hitters.drop (['Salary', 'League', 'Division', 'NewLeague'], axis = 1).astype('float64')
X = pd.concat([X_prep,  dummies[['League_A', 'Division_E', 'NewLeague_A']]], axis=1)
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.66)


In [ ]:
# the DL model is similar to other models implemeted in sklearn. 
# we first define the model, then fit the model, and finally predict the result

"""
I am listing out the hyperameters to be tuned. 
From this simple example with only one layer, we could get a sense of the number of hyperameters in NN.

Actually the number of the hyperameters gets exponentially larger as the number of layers increases.
"""
# define the model.model.add
dropout_rate = 0.4
first_layout = 50
epochs = 150
batch_size = 32
activation = 'relu'
loss = 'mean_squared_error'
optimizer = 'rmsprop'
metrics = ['mae']

model = Sequential()
model.add(Dropout(rate=dropout_rate, input_shape=(X_train.shape[1],)))
model.add(Dense(first_layout, activation=activation))
model.add(Dense(1))
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
# we can use the model.summary() to see the structure of the model
print(model.summary())



In [ ]:
# fit the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
print(model.summary())

In [ ]:
# evaluate the model
# model.evaluate returns the loss value & metrics values for the model in test mode.
mse_test, mae_test = model.evaluate(X_test, y_test)
print('Test mse: %.3f, Test mae: %.3f' % (mse_test, mae_test))

## 10.9.2 A Multilayer Network on the MNIST Digit Data

In [ ]:
# we could load the MNIST data set from keras.datasets
# keras.datasets also contains other well-known datasets, such as cifar10, fashion_mnist, etc.
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
# since the data set if for image, so each image is a 28*28 matrix.
print(X_train.shape)
print(np.max(X_train))

# the y_train the group label for the training data
print(y_train.shape)
print(np.unique(y_train))

In [ ]:
# let us plot some of the images
for i in range(9):
	# define subplot
	plt.subplot(330 + 1 + i)
	plt.imshow(X_train[i], cmap=plt.get_cmap('gray'))
plt.show()

In [ ]:
# now let us reshape each image (i.e. matrix) to a vector
X_train = X_train.reshape((X_train.shape[0], 28*28)).astype('float32')
X_test = X_test.reshape((X_test.shape[0], 28*28)).astype('float32')
""" 
We know that each pixel has its unique color code and also we know that it has a maximum value of 255. 
To perform Machine Learning, it is important to convert all the values from 0 to 255 for every pixel to 
a range of values from 0 to 1. The simplest way is to divide the value of every pixel by 255 to get the 
values in the range of 0 to 1.
"""
X_train = X_train / 255
X_test = X_test / 255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
model = Sequential()
model.add(Dense(256, input_dim = 28 * 28, activation= 'relu'))
model.add(Dropout(rate=0.4))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(10, activation = 'softmax'))

In [ ]:
# compile the model and fit the model 
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history_model = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test), verbose=0)
_, acc = model.evaluate(X_test, y_test)
print('Test accuracy: %.3f' % acc)

In [ ]:
# we can plot the metric history 
history_dict = history_model.history
plt.title('Classification Accuracy')
plt.plot(history_dict['accuracy'], color='black', label='train')
plt.plot(history_dict['val_accuracy'], color='orange', label='test')
plt.legend(loc='best')
plt.show()
""" 
One thing to notice is the test accuracy is actually higher than the training accuracy. 
It is kind of uncommon - if you have a good explaination on this, let me know. 
"""

## 10.9.3 Convolutional Neural Networks

In [ ]:
(X_train, y_train), (X_test, y_test) = tk.datasets.cifar100.load_data()

In [ ]:
# take a look at the data shape, compare to MNIST, this dataset is also image but with 
# different channels, i.e. squared image with 32×32 pixels and three color channels
print(X_train.shape)
print(np.max(X_train))

# the y_train the group label for the training data
print(y_train.shape)
print(np.unique(y_train))

In [ ]:
# show a few samples
for i in range(9):
	# define subplot
	plt.subplot(330 + 1 + i)
	plt.imshow(X_train[i], cmap=plt.get_cmap('gray'))
plt.show()

In [ ]:
# the workflow is similar to before
# we prepare the data 
# then define the model architecture 
# then fit the model 
# then exam the model performance 

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255
X_test = X_test / 255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
# this is a 3 block (each block contains conv layer and pooling layer) VGG architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))

# then define the output classifer part of the model 
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(100, activation='softmax'))

In [ ]:
# compile the model and fit the model 
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history_model = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_test, y_test), verbose=0)
_, acc = model.evaluate(X_test, y_test)
print('Test accuracy: %.3f' % acc)

In [ ]:
# we can plot the metric history 
history_dict = history_model.history
plt.title('Classification Accuracy')
plt.plot(history_dict['accuracy'], color='black', label='train')
plt.plot(history_dict['val_accuracy'], color='orange', label='test')
plt.legend(loc='best')
plt.show()

In [ ]:
# as before, we can add in dropout 
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='softmax'))

# compile the model and fit the model 
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history_model = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_test, y_test), verbose=0)
_, acc = model.evaluate(X_test, y_test)
print('Test accuracy: %.3f' % acc)
""" 
From the result, we can see that adding in dropout decreases the test accuracy from 
37.6% to 34.1%. This is mainly due to the underfit of the model.

Dropout helps in the case of overfitting, to see the benefit of the dropout, we would need to 
increase epochs to a large number (~ 1000). 
"""

In [ ]:
"""
Some other regularization technique is not mentioned in this lab is weight decay (i.e. similar to the concept of Lasso and Ridge)
In this setup, we can add regularization to the weights using the syntax 'kernel_regularizer'
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))

Some other regularization methods are data augmentation and early stopping. 
"""